<a href="https://colab.research.google.com/github/devikaajay/DS_Assessment_2/blob/main/DSA_Assessment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import numpy as np

In [2]:
train_df = pd.read_csv('train_LZdllcl.csv')
test_df = pd.read_csv('test_2umaH9m.csv')
sample_submission_df = pd.read_csv('sample_submission_M0L0uXE.csv')

Exploratory Data Analysis

In [3]:
print("\nDistribution of 'is_promoted' in Training Data:")
print(train_df['is_promoted'].value_counts())
print(train_df['is_promoted'].value_counts(normalize=True) * 100)


Distribution of 'is_promoted' in Training Data:
is_promoted
0    50140
1     4668
Name: count, dtype: int64
is_promoted
0    91.482995
1     8.517005
Name: proportion, dtype: float64


Pre Processing

In [4]:
train_df_processed = train_df.copy()
test_df_processed = test_df.copy()

In [5]:
mode_education_train = train_df_processed['education'].mode()[0]
train_df_processed['education'].fillna(mode_education_train, inplace=True)
test_df_processed['education'].fillna(mode_education_train, inplace=True)

mode_prev_rating_train = train_df_processed['previous_year_rating'].mode()[0]
train_df_processed['previous_year_rating'].fillna(mode_prev_rating_train, inplace=True)
test_df_processed['previous_year_rating'].fillna(mode_prev_rating_train, inplace=True)

<ipython-input-5-11361faa13ea>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df_processed['education'].fillna(mode_education_train, inplace=True)
<ipython-input-5-11361faa13ea>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

In [6]:
print("\nMissing values after imputation (Train DataFrame):")
print(train_df_processed.isnull().sum()[train_df_processed.isnull().sum() > 0])

print("\nMissing values after imputation (Test DataFrame):")
print(test_df_processed.isnull().sum()[test_df_processed.isnull().sum() > 0])


Missing values after imputation (Train DataFrame):
Series([], dtype: int64)

Missing values after imputation (Test DataFrame):
Series([], dtype: int64)


In [7]:
train_employee_ids = train_df_processed['employee_id']
test_employee_ids = test_df_processed['employee_id']

X_train = train_df_processed.drop(columns=['employee_id', 'is_promoted'])
y_train = train_df_processed['is_promoted']

X_test = test_df_processed.drop(columns=['employee_id'])

categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
numerical_cols = [col for col in X_train.columns if X_train[col].dtype != 'object']


In [8]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_encoded_array = encoder.fit_transform(X_train[categorical_cols])
X_test_encoded_array = encoder.transform(X_test[categorical_cols])
encoded_feature_names = encoder.get_feature_names_out(categorical_cols)

X_train_categorical_df = pd.DataFrame(X_train_encoded_array, columns=encoded_feature_names, index=X_train.index)
X_test_categorical_df = pd.DataFrame(X_test_encoded_array, columns=encoded_feature_names, index=X_test.index)

train_cols_set = set(X_train_categorical_df.columns)
test_cols_set = set(X_test_categorical_df.columns)

missing_in_test = list(train_cols_set - test_cols_set)
missing_in_train = list(test_cols_set - train_cols_set)

In [9]:
for col in missing_in_test:
    X_test_categorical_df[col] = 0
for col in missing_in_train:
    X_train_categorical_df[col] = 0

X_test_categorical_df = X_test_categorical_df[X_train_categorical_df.columns]

X_train_final = pd.concat([X_train[numerical_cols], X_train_categorical_df], axis=1)
X_test_final = pd.concat([X_test[numerical_cols], X_test_categorical_df], axis=1)

In [10]:
print("\nShape of X_train_final after One-Hot Encoding:", X_train_final.shape)
print("Shape of X_test_final after One-Hot Encoding:", X_test_final.shape)
print("\nFirst 5 rows of X_train_final:")
print(X_train_final.head())
print("\nFirst 5 rows of X_test_final:")
print(X_test_final.head())


Shape of X_train_final after One-Hot Encoding: (54808, 58)
Shape of X_test_final after One-Hot Encoding: (23490, 58)

First 5 rows of X_train_final:
   no_of_trainings  age  previous_year_rating  length_of_service  \
0                1   35                   5.0                  8   
1                1   30                   5.0                  4   
2                1   34                   3.0                  7   
3                2   39                   1.0                 10   
4                1   45                   3.0                  2   

   KPIs_met >80%  awards_won?  avg_training_score  department_Analytics  \
0              1            0                  49                   0.0   
1              0            0                  60                   0.0   
2              0            0                  50                   0.0   
3              0            0                  50                   0.0   
4              0            0                  73                 

Modelling

In [11]:
model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced', n_jobs=-1)
print("Training RandomForestClassifier model...")
model.fit(X_train_final, y_train)
print("Model training complete.")

Training RandomForestClassifier model...
Model training complete.


In [12]:
model_base = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

In [13]:
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
f1_scorer = make_scorer(f1_score, average='binary')
grid_search = GridSearchCV(estimator=model_base, param_grid=param_grid,cv=3, scoring=f1_scorer, verbose=1, n_jobs=-1)
grid_search.fit(X_train_final, y_train)
best_model = grid_search.best_estimator_
print(f"Best hyperparameters found: {grid_search.best_params_}")
print(f"Best F1 Score on validation sets during tuning: {grid_search.best_score_:.4f}")

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best hyperparameters found: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 100}
Best F1 Score on validation sets during tuning: 0.4555


Prediction

In [14]:
print("Making predictions on the pre-processed test dataset")
predictions = best_model.predict(X_test_final)
print("Predictions generated.")

Making predictions on the pre-processed test dataset...
Predictions generated.


In [15]:
sample_submission_df['is_promoted'] = predictions
output_filename = 'submission_hr_analytics.csv'
sample_submission_df.to_csv(output_filename, index=False)
print(f"\nSubmission file saved successfully as '{output_filename}' in your current directory.")
print("\nFinal submission file head:")
print(sample_submission_df.head())


Submission file saved successfully as 'submission_hr_analytics.csv' in your current directory.

Final submission file head:
   employee_id  is_promoted
0         8724            0
1        74430            0
2        72255            0
3        38562            0
4        64486            0
